# Minimal chromospyce example

In [ ]:
import chromospyce
import numpy as np

BINS_NUM = 1000

# Step 1: Generate random structure, returns a 2D numpy array:
def make_random_3D_chromatin_structure(n):
    position = np.array([0.0, 0.0, 0.0])
    positions = [position.copy()]
    for _ in range(n):
        step = np.random.choice([-1.0, 0.0, 1.0], size=3)  # Randomly choose to move left, right, up, down, forward, or backward
        position += step
        positions.append(position.copy())
    return np.array(positions)

random_structure = make_random_3D_chromatin_structure(BINS_NUM)

# Step 2: Display the structure in a chromospyce widget, using the default viewconfig
chromospyce.Widget(random_structure)

In [ ]:
import pandas as pd

# Generate completely random positions
BINS_NUM = 100
df = pd.DataFrame(np.random.rand(BINS_NUM,3), columns=list('xyz'))

numbers = list(range(0, BINS_NUM+1))
vc = {
    # "color": "purple",
    "color": {
        "values": numbers,
        "min": 0.0,
        "max": BINS_NUM,
        "colorScale": "Spectral"
    }, 
    "scale": 0.01, 
    "links": True, 
    "mark": "sphere"
}
chromospyce.Widget(df, vc)